### ***Первая лабораторная***

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
name = "FirstProject"
conf = SparkConf().setAppName(name).setMaster('local[1]')
sc = SparkContext(conf=conf)

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [2]:
#Проверяем, что все нужные файлы были скопированы
!hadoop fs -ls /data

Found 7 items
-rwxr-xr-x   3 root root   79500408 2021-11-10 15:00 /data/nyctaxi.csv
-rwxr-xr-x   3 root root         12 2021-11-10 14:57 /data/one.txt
-rwxr-xr-x   3 root root   74162295 2021-11-10 15:05 /data/posts_sample.xml
-rwxr-xr-x   3 root root      40269 2021-11-10 15:05 /data/programming-languages.csv
-rwxr-xr-x   3 root root       5647 2021-11-10 15:00 /data/station.csv
-rwxr-xr-x   3 root root   80208848 2021-11-10 15:00 /data/trip.csv
-rwxr-xr-x   3 root root    5315699 2021-11-10 15:01 /data/warandsociety.txt


In [3]:
sc

<SparkContext master=local[1] appName=FirstProject>

In [3]:
#создаем RDD для текстового файла warandpeace.txt
warandpeace = sc.textFile("/mnt/data/warandsociety.txt")
#выводим количество строк
warandpeace.count()

12851

In [4]:
#считываем первые 10 строк
warandpeace.take(10)

['Лев Николаевич Толстой',
 'Война и мир. Книга 1',
 '',
 'Война и мир – 1',
 '',
 ' ',
 ' http://www.lib.ru',
 '',
 'Аннотация ',
 '']

In [5]:
#получаем первую строку
warandpeace.first()

'Лев Николаевич Толстой'

In [6]:
#ищем строки, в которых содержится слово "война"
linesWithWar = warandpeace.filter(lambda x: x.__contains__("война"))
linesWithWar

PythonRDD[5] at RDD at PythonRDD.scala:53

In [7]:
#выводим количество этих строк
linesWithWar.cache()
linesWithWar.count()
linesWithWar.count()

54

In [9]:
#ищем гистограмму слов и затем сохраняем в файл warandpeace_histogram
wordsCounts=warandpeace.flatMap(lambda line1: line1.split(" "))
wordsCounts2=wordsCounts.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
wordsCounts2.saveAsTextFile("/mnt/data/warandpeace_histogram.txt")

#### Операции с множествами

In [10]:
#Объединение двух множеств
a = sc.parallelize([1,2,3,4])
b = sc.parallelize([3,4,6,7])
a.union(b).collect()

[1, 2, 3, 4, 3, 4, 6, 7]

In [11]:
#Убираем дубликаты
a.union(b).distinct().collect()

[2, 4, 6, 1, 3, 7]

In [12]:
#Пересечение множеств
a.intersection(b).collect()

[4, 3]

In [13]:
#разность множеств
a.subtract(b).collect()

[2, 1]

In [14]:
#широковещательные переменные
broadcastVar = sc.broadcast([1,2,3])

In [15]:
broadcastVar.value

[1, 2, 3]

#### Работаем с данными о такси

In [16]:
#Считываем данные
taxi = sc.textFile("/mnt/data/nyctaxi.csv")
#выводим первые пять строк
taxi.take(5)

['"_id","_rev","dropoff_datetime","dropoff_latitude","dropoff_longitude","hack_license","medallion","passenger_count","pickup_datetime","pickup_latitude","pickup_longitude","rate_code","store_and_fwd_flag","trip_distance","trip_time_in_secs","vendor_id"',
 '"29b3f4a30dea6688d4c289c9672cb996","1-ddfdec8050c7ef4dc694eeeda6c4625e","2013-01-11 22:03:00",+4.07033460000000E+001,-7.40144200000000E+001,"A93D1F7F8998FFB75EEF477EB6077516","68BC16A99E915E44ADA7E639B4DD5F59",2,"2013-01-11 21:48:00",+4.06760670000000E+001,-7.39810790000000E+001,1,,+4.08000000000000E+000,900,"VTS"',
 '"2a80cfaa425dcec0861e02ae44354500","1-b72234b58a7b0018a1ec5d2ea0797e32","2013-01-11 04:28:00",+4.08190960000000E+001,-7.39467470000000E+001,"64CE1B03FDE343BB8DFB512123A525A4","60150AA39B2F654ED6F0C3AF8174A48A",1,"2013-01-11 04:07:00",+4.07280540000000E+001,-7.40020370000000E+001,1,,+8.53000000000000E+000,1260,"VTS"',
 '"29b3f4a30dea6688d4c289c96758d87e","1-387ec30eac5abda89d2abefdf947b2c1","2013-01-11 22:02:00",+4.0727

In [17]:
#топ 10 популярных номеров такси
#избавляемся от первой строки с заголовками
indexes_taxi = taxi.first()
taxi = taxi.filter(lambda row: row != indexes_taxi)
taxi.take(4)

['"29b3f4a30dea6688d4c289c9672cb996","1-ddfdec8050c7ef4dc694eeeda6c4625e","2013-01-11 22:03:00",+4.07033460000000E+001,-7.40144200000000E+001,"A93D1F7F8998FFB75EEF477EB6077516","68BC16A99E915E44ADA7E639B4DD5F59",2,"2013-01-11 21:48:00",+4.06760670000000E+001,-7.39810790000000E+001,1,,+4.08000000000000E+000,900,"VTS"',
 '"2a80cfaa425dcec0861e02ae44354500","1-b72234b58a7b0018a1ec5d2ea0797e32","2013-01-11 04:28:00",+4.08190960000000E+001,-7.39467470000000E+001,"64CE1B03FDE343BB8DFB512123A525A4","60150AA39B2F654ED6F0C3AF8174A48A",1,"2013-01-11 04:07:00",+4.07280540000000E+001,-7.40020370000000E+001,1,,+8.53000000000000E+000,1260,"VTS"',
 '"29b3f4a30dea6688d4c289c96758d87e","1-387ec30eac5abda89d2abefdf947b2c1","2013-01-11 22:02:00",+4.07277180000000E+001,-7.39942860000000E+001,"2D73B0C44F1699C67AB8AE322433BDB7","6F907BC9A85B7034C8418A24A0A75489",5,"2013-01-11 21:46:00",+4.07577480000000E+001,-7.39649810000000E+001,1,,+3.01000000000000E+000,960,"VTS"',
 '"2a80cfaa425dcec0861e02ae446226e4","1

In [18]:
#разбиваем строки на массивы строк, которые разделены запятыми
taxiParse = taxi.map(lambda line: line.split(","))

In [19]:
#преобразуем массив строк в массив пар-ключей
taxiMedKey = taxiParse.map(lambda row: (row[6], 1))
taxiMedCounts = taxiMedKey.reduceByKey(lambda v1, v2: v1+v2)
#taxiMedCounts.take(taxiMedCounts.count())

In [20]:
#Тоже самое одной строкой
taxiCounts = taxi.map(lambda line:line.split(",")).map(lambda row:(row[6],1)).reduceByKey(lambda a,b: a+b)
type(taxiCounts)

pyspark.rdd.PipelinedRDD

In [21]:
#получаем топ-10 
taxiCounts.map(lambda x: x[::-1]).top(10)

[(44, '"AB44AD9A03B7CFAF3925103BDCC0AF23"'),
 (41, '"71CACFBADF9568AAE88A843DB511D172"'),
 (41, '"6483B9BFCB216EC88986EA3AB13064E7"'),
 (41, '"4C73459B430339981D78795300433438"'),
 (40, '"67E71D24AF704D814A0A825005ADA72E"'),
 (40, '"02E5A4136FD0A775A023A005A4EABC62"'),
 (39, '"9DFBCD218E7116F34C044F0680A0FB8A"'),
 (39, '"8DEB70907D00AA1D7FF5E2683240549B"'),
 (39, '"7989C2AB3F345F4AB54D3CF1E0480D67"'),
 (39, '"6C9F67DF658DC5636F9E7752F203F70A"')]

#### Работаем с данными о поездках

In [22]:
#Создаем два RDD на основе таблиц и выводим их титульники
tripD = sc.textFile("/mnt/data/trip.csv")
stationD = sc.textFile("/mnt/data/station.csv")

In [23]:
tripHeader=tripD.first()
tripHeader

'id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code'

In [24]:
stationHeader=stationD.first()
stationHeader

'id,name,lat,long,dock_count,city,installation_date'

In [25]:
#избавляемся от первой строки с заглавиями
trips= tripD.filter(lambda row: row !=tripHeader).map(lambda row: row.split(","))
stations=stationD.filter(lambda row: row !=stationHeader).map(lambda row: row.split(","))

In [26]:
trips.take(2)

[['4576',
  '63',
  '8/29/2013 14:13',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '70',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138']]

In [27]:
stations.take(2)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013'],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '8/5/2013']]

In [28]:
#Ниже, после объединения получались нули, поэтому я решила попробовать 
#проиндексировать не по номеру велостоянки, а так же, по её названию
stationsIndexed= stations.keyBy(lambda row: (row[1]))
stationsIndexed.take(2)

[('San Jose Diridon Caltrain Station',
  ['2',
   'San Jose Diridon Caltrain Station',
   '37.329732',
   '-121.90178200000001',
   '27',
   'San Jose',
   '8/6/2013']),
 ('San Jose Civic Center',
  ['3',
   'San Jose Civic Center',
   '37.330698',
   '-121.888979',
   '15',
   'San Jose',
   '8/5/2013'])]

In [29]:
#Индексируем по начальной и конечной станциям
tripsStart = trips.keyBy(lambda row: row[3])
tripsEnd = trips.keyBy(lambda row: row[6])
tripsStart.take(2), tripsEnd.take(2)

([('South Van Ness at Market',
   ['4576',
    '63',
    '8/29/2013 14:13',
    'South Van Ness at Market',
    '66',
    '8/29/2013 14:14',
    'South Van Ness at Market',
    '66',
    '520',
    'Subscriber',
    '94127']),
  ('San Jose City Hall',
   ['4607',
    '70',
    '8/29/2013 14:42',
    'San Jose City Hall',
    '10',
    '8/29/2013 14:43',
    'San Jose City Hall',
    '10',
    '661',
    'Subscriber',
    '95138'])],
 [('South Van Ness at Market',
   ['4576',
    '63',
    '8/29/2013 14:13',
    'South Van Ness at Market',
    '66',
    '8/29/2013 14:14',
    'South Van Ness at Market',
    '66',
    '520',
    'Subscriber',
    '94127']),
  ('San Jose City Hall',
   ['4607',
    '70',
    '8/29/2013 14:42',
    'San Jose City Hall',
    '10',
    '8/29/2013 14:43',
    'San Jose City Hall',
    '10',
    '661',
    'Subscriber',
    '95138'])])

In [30]:
#объединяем по ключу
startTrips = stationsIndexed.join(tripsStart)
endTrips = stationsIndexed.join(tripsEnd)

In [31]:
#выводим полученные графы
startTrips.toDebugString()

b'(4) PythonRDD[86] at RDD at PythonRDD.scala:53 []\n |  MapPartitionsRDD[78] at mapPartitions at PythonRDD.scala:145 []\n |  ShuffledRDD[77] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(4) PairwiseRDD[76] at join at <ipython-input-30-9f06234f33cb>:2 []\n    |  PythonRDD[75] at join at <ipython-input-30-9f06234f33cb>:2 []\n    |  UnionRDD[74] at union at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[72] at RDD at PythonRDD.scala:53 []\n    |  /mnt/data/station.csv MapPartitionsRDD[64] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /mnt/data/station.csv HadoopRDD[63] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[73] at RDD at PythonRDD.scala:53 []\n    |  /mnt/data/trip.csv MapPartitionsRDD[62] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /mnt/data/trip.csv HadoopRDD[61] at textFile at NativeMethodAccessorImpl.java:0 []'

In [32]:
endTrips.toDebugString()

b'(4) PythonRDD[87] at RDD at PythonRDD.scala:53 []\n |  MapPartitionsRDD[85] at mapPartitions at PythonRDD.scala:145 []\n |  ShuffledRDD[84] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(4) PairwiseRDD[83] at join at <ipython-input-30-9f06234f33cb>:3 []\n    |  PythonRDD[82] at join at <ipython-input-30-9f06234f33cb>:3 []\n    |  UnionRDD[81] at union at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[79] at RDD at PythonRDD.scala:53 []\n    |  /mnt/data/station.csv MapPartitionsRDD[64] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /mnt/data/station.csv HadoopRDD[63] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[80] at RDD at PythonRDD.scala:53 []\n    |  /mnt/data/trip.csv MapPartitionsRDD[62] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /mnt/data/trip.csv HadoopRDD[61] at textFile at NativeMethodAccessorImpl.java:0 []'

In [33]:
startTrips.count(),endTrips.count()

(658617, 657435)

#### Объявление классов

In [34]:
from typing import NamedTuple
from datetime import datetime

class Trip(NamedTuple):
    tripId: int
    duration: int
    startDate: str
    startStation: str
    startTerminal: int
    endDate: str
    endStation: str
    endTerminal: int
    bikeId: int
    subscriptionType: str
    zipCode: str
        
    
    def add(row):
        return Trip(                             
             tripId=int(row[0]),
             duration=int(row[1]),
             startDate=datetime.strptime(row[2], '%m/%d/%Y %H:%M'),
             startStation=row[3],
             startTerminal=int(row[4]),
             endDate=datetime.strptime(row[5], '%m/%d/%Y %H:%M'),
             endStation=row[6],
             endTerminal=row[7],
             bikeId=int(row[8]),
             subscriptionType=row[9],
             zipCode=row[10]
        )

In [35]:
tripsInternal = trips.map(Trip.add)

In [36]:
tripsInternal.first()

Trip(tripId=4576, duration=63, startDate=datetime.datetime(2013, 8, 29, 14, 13), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 14, 14), endStation='South Van Ness at Market', endTerminal='66', bikeId=520, subscriptionType='Subscriber', zipCode='94127')

In [37]:
class Station(NamedTuple):
    station_id: int
    name: str
    lat: float
    long: float
    dockcount: int
    landmark: str
    installation: str
    

    
    def add(row):
        return Station(
            station_id=int(row[0]),
            name=row[1],
            lat=float(row[2]),
            long=float(row[3]),
            dockcount=int(row[4]),
            landmark=row[5],
            installation=datetime.strptime(row[6], '%m/%d/%Y'),
            
        )

In [38]:
stationsInternal = stations.map(Station.add)

In [39]:
stationsInternal.take(2)

[Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
 Station(station_id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 5, 0, 0))]

In [40]:
stationsInternal.first().name

'San Jose Diridon Caltrain Station'

In [41]:
#считаем среднее время поездки
#сначала получаем коллекцию пар ключей
#Объединяем по начальной станции
tripsByStartStation = tripsInternal.keyBy(lambda record: record.startStation)

In [42]:
tripsByStartStation.first()

('South Van Ness at Market',
 Trip(tripId=4576, duration=63, startDate=datetime.datetime(2013, 8, 29, 14, 13), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 14, 14), endStation='South Van Ness at Market', endTerminal='66', bikeId=520, subscriptionType='Subscriber', zipCode='94127'))

In [43]:
from functools import reduce
#Рассчитываем среднее время поездки для каждого стартового парковочного места
avgDurationByStartStation = tripsByStartStation\
 .mapValues(lambda trip: trip.duration)\
 .groupByKey()\
 .mapValues(lambda values: reduce(lambda v1, v2: (v1 + v2) , values) / len(values))

In [44]:
avgDurationByStartStation.map(lambda x: x[::-1]).top(10)

[(7090.239417989418, 'University and Emerson'),
 (4628.005847953216, 'California Ave Caltrain Station'),
 (4579.234741784037, 'Redwood City Public Library'),
 (4438.1613333333335, 'Park at Olive'),
 (4208.016938519448, 'San Jose Civic Center'),
 (4174.082373782108, 'Rengstorff Avenue / California Street'),
 (3959.491961414791, 'Redwood City Medical Center'),
 (3210.6489815253435, 'Palo Alto Caltrain Station'),
 (2716.7700348432054, 'San Mateo County Center'),
 (2481.2537313432836, 'Broadway at Main')]

In [45]:
#Это тоже самое делаем но с помощью другого метода
avgDurationByStartStation2 = tripsByStartStation.mapValues(lambda x:x.duration)\
  .aggregateByKey((0,0),lambda acc, value: (acc._1 + value, acc._2 + 1),
   lambda acc1, acc2: (acc1._1+acc2._1, acc1._2+acc2._2))\
  .mapValues(lambda acc:acc._1/acc._2)


In [46]:
avgDurationByStartStation.map(lambda x: x[::-1]).top(10)

[(7090.239417989418, 'University and Emerson'),
 (4628.005847953216, 'California Ave Caltrain Station'),
 (4579.234741784037, 'Redwood City Public Library'),
 (4438.1613333333335, 'Park at Olive'),
 (4208.016938519448, 'San Jose Civic Center'),
 (4174.082373782108, 'Rengstorff Avenue / California Street'),
 (3959.491961414791, 'Redwood City Medical Center'),
 (3210.6489815253435, 'Palo Alto Caltrain Station'),
 (2716.7700348432054, 'San Mateo County Center'),
 (2481.2537313432836, 'Broadway at Main')]

In [47]:
#ищем первую поездку для каждой стоянки
#группируем по ключу
firstGrouped = tripsByStartStation\
    .groupByKey()\
    .mapValues(lambda values: sorted(values, key=lambda x: x.startDate ))

In [48]:
firstGrouped.map(lambda x: x[::-1]).first()

([Trip(tripId=4073, duration=1067, startDate=datetime.datetime(2013, 8, 29, 9, 24), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 9, 42), endStation='San Francisco Caltrain 2 (330 Townsend)', endTerminal='69', bikeId=321, subscriptionType='Subscriber', zipCode='94703'),
  Trip(tripId=4075, duration=1117, startDate=datetime.datetime(2013, 8, 29, 9, 24), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 9, 43), endStation='San Francisco Caltrain 2 (330 Townsend)', endTerminal='69', bikeId=316, subscriptionType='Subscriber', zipCode='94122'),
  Trip(tripId=4074, duration=1131, startDate=datetime.datetime(2013, 8, 29, 9, 24), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 9, 43), endStation='San Francisco Caltrain 2 (330 Townsend)', endTerminal='69', bikeId=317, subscriptionType='Subscriber', zipCode='94115'),
  Trip(tripId=4076, duration=11

In [49]:
#Тоже самое, но с помощью reduceByKey
firstGrouped  = tripsByStartStation\
    .reduceByKey(
        lambda trip1, trip2: trip1 if trip1.startDate < trip2.startDate else trip2)

In [50]:
firstGrouped.first()

('South Van Ness at Market',
 Trip(tripId=4074, duration=1131, startDate=datetime.datetime(2013, 8, 29, 9, 24), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 9, 43), endStation='San Francisco Caltrain 2 (330 Townsend)', endTerminal='69', bikeId=317, subscriptionType='Subscriber', zipCode='94115'))

### Сама лабораторная работа

#### Ищем велоcипед с максимальным пробегом

In [51]:
#группируем по id
Bike = tripsInternal.keyBy(lambda trip: trip.bikeId)
Bike.first()

(520,
 Trip(tripId=4576, duration=63, startDate=datetime.datetime(2013, 8, 29, 14, 13), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 14, 14), endStation='South Van Ness at Market', endTerminal='66', bikeId=520, subscriptionType='Subscriber', zipCode='94127'))

In [52]:
#потом получаем продолжительности и скадываем их для каждой поездки
BikeDuration = Bike\
    .mapValues(lambda trip: trip.duration)\
    .reduceByKey(lambda trip1, trip2: trip1 + trip2)

In [53]:
#Выводим топ 10
BikeDuration=BikeDuration.map(lambda x: x[::-1]).top(1)[0][1]
BikeDuration

535

#### Найти наибольшее расстояние между станциями.

In [54]:
#Сначала делаем пары из станций (начальная, конечная) 
#и продолжительности поездки между ними
Trips2 = tripsInternal\
    .filter(lambda trip: trip.startStation != trip.endStation)\
    .keyBy(lambda trip: (trip.startStation, trip.endStation))\
    .mapValues(lambda trip: trip.duration)

In [55]:
Trips2.take(2)

[(('South Van Ness at Market', 'Market at 10th'), 83),
 (('Santa Clara at Almaden', 'Adobe on Almaden'), 109)]

In [56]:
#Затем рассчитываем среднее для каждой комбинации начальной и конечной станций
Trips2 = Trips2.aggregateByKey((0, 0),\
        lambda acc, value: (acc[0] + value, acc[1] + 1),\
        lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]),)\
        .mapValues(lambda values: values[0] / values[1])

In [57]:
#Наибольшее расстояние
Trips2.map(lambda x: x[::-1]).top(1)

[(229914.0, ('Redwood City Medical Center', 'SJSU - San Salvador at 9th'))]

#### Найти путь велосипеда с максимальным пробегом через станции.

In [58]:
MaxDuration = tripsInternal.filter(lambda trip: trip.bikeId == BikeDuration)\
    .sortBy(lambda trip: trip.startDate).map(lambda trip:trip.startStation)    

In [59]:
#Выводим последовательность станций по которым он проехал
MaxDuration.collect()

['Post at Kearney',
 'San Francisco Caltrain (Townsend at 4th)',
 'San Francisco Caltrain 2 (330 Townsend)',
 'Market at Sansome',
 '2nd at Townsend',
 'San Francisco City Hall',
 'Civic Center BART (7th at Market)',
 'Post at Kearney',
 'Embarcadero at Sansome',
 'Washington at Kearney',
 'Market at Sansome',
 'Market at Sansome',
 '2nd at Folsom',
 'Temporary Transbay Terminal (Howard at Beale)',
 '2nd at Townsend',
 'Embarcadero at Sansome',
 'Clay at Battery',
 'Harry Bridges Plaza (Ferry Building)',
 'Clay at Battery',
 'San Francisco Caltrain (Townsend at 4th)',
 'Steuart at Market',
 '2nd at Townsend',
 'Harry Bridges Plaza (Ferry Building)',
 'Townsend at 7th',
 'San Francisco Caltrain 2 (330 Townsend)',
 'San Francisco Caltrain 2 (330 Townsend)',
 'Steuart at Market',
 'San Francisco Caltrain (Townsend at 4th)',
 '2nd at South Park',
 'Post at Kearney',
 '2nd at Folsom',
 'Mechanics Plaza (Market at Battery)',
 'Powell at Post (Union Square)',
 'Powell at Post (Union Square)',

#### Найти количество велосипедов в системе

In [60]:
number_bicycles = tripsInternal.keyBy(lambda r: r.bikeId).groupByKey()

In [61]:
number_bicycles.count()

700

#### Найти пользователей потративших на поездки более 3 часов.

In [62]:
#Я не нашла данные о пользователе, так что, наверное, можно использовать
#id поездки

In [63]:
Id_trip = tripsInternal.filter(lambda trip: trip.duration >180)\
    .map(lambda trip: trip.tripId)

In [64]:
Id_trip.collect()

[5088,
 4721,
 4636,
 4812,
 4720,
 4798,
 4944,
 4400,
 4606,
 4705,
 4841,
 4704,
 4824,
 4854,
 4502,
 4638,
 4855,
 4994,
 4459,
 5074,
 4081,
 4337,
 4401,
 4411,
 4467,
 4765,
 5001,
 5002,
 4560,
 4592,
 4673,
 4714,
 4748,
 4317,
 4559,
 4880,
 4668,
 4667,
 4443,
 5105,
 4341,
 4423,
 4911,
 4504,
 4584,
 4599,
 4801,
 4870,
 4743,
 4799,
 5006,
 4686,
 4906,
 4177,
 4436,
 5057,
 4084,
 4595,
 4377,
 4600,
 4959,
 4828,
 4450,
 4558,
 4657,
 4796,
 4331,
 4280,
 4290,
 4876,
 4766,
 4871,
 5078,
 4744,
 4376,
 4392,
 4602,
 5100,
 4448,
 4722,
 4850,
 4615,
 4582,
 4640,
 4805,
 4715,
 4941,
 4454,
 4647,
 4750,
 4393,
 4822,
 4300,
 4968,
 4890,
 5075,
 4570,
 4278,
 4399,
 4660,
 4745,
 4955,
 4785,
 4730,
 4786,
 4447,
 5032,
 4312,
 4695,
 4700,
 4875,
 4659,
 4998,
 4496,
 4716,
 4527,
 4612,
 4204,
 4731,
 4883,
 4517,
 4429,
 4699,
 4434,
 4408,
 4961,
 5099,
 4725,
 4347,
 5095,
 5102,
 4414,
 5050,
 4334,
 4385,
 4981,
 5062,
 4970,
 4458,
 4879,
 4999,
 4751,
 4996,